In [1]:
import keras_tuner
from tensorflow import keras
from keras_tuner.tuners import RandomSearch
import pandas as pd

Using TensorFlow backend


2023-12-13 11:28:55.233131: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-13 11:28:55.269182: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 11:28:55.269228: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 11:28:55.269277: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 11:28:55.279610: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-13 11:28:55.280499: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
#adv_df = pd.read_csv("./data/shadow_model_outputs.csv")
adv_df = pd.read_csv("data/shadow_model_outputs_proba.csv")
adv_ddf_shuffled = adv_df.sample(frac=1, random_state=1).reset_index(drop=True).drop(columns=["Unnamed: 0"])
adv_y = adv_ddf_shuffled["y"]
adv_X = adv_ddf_shuffled.drop(columns=["y"])

In [3]:
adv_df_test = pd.read_csv("data/shadow_model_outputs_proba_test_set.csv")
#adv_df_test = pd.read_csv("./data/shadow_model_outputs_test_set.csv")
adv_ddf_shuffled_test = adv_df_test.sample(frac=1, random_state=1).reset_index(drop=True).drop(columns=["Unnamed: 0"])
adv_y_test = adv_ddf_shuffled_test["y"]
adv_X_test = adv_ddf_shuffled_test.drop(columns=["y"])

In [4]:
def build_model(hp):
    model = keras.Sequential()
    #model.add(keras.layers.Dense(units=8000, activation="relu"))
    #if hp.Boolean("lstm"):
        #model.add(keras.layers.Dropout(rate=0.1))
    #    model.add(keras.layers.Reshape((adv_X.shape[1],1)))
    #    model.add(keras.layers.LSTM(hp.Int('units_lstm', min_value=2, max_value=30, step=4), activation='relu', return_sequences=False, return_state=False))
    #else:
    #    model.add(keras.layers.Dense(units=hp.Int('units_dense1', min_value=8, max_value=64, step=8)))
    
    #model.add(keras.layers.Dense(units=hp.Int('units_dense1', min_value=45, max_value=47, step=1), activation="relu"))
    
    model.add(keras.layers.Dense(46, activation="relu"))
    
    model.add(keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.01, max_value=0.04, step=0.01)))
    
    #if hp.Boolean("dropout"):
    #model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.Dense(18, activation="relu"))
    #model.add(keras.layers.Dense(units=hp.Int('units_dense2', min_value=17, max_value=23, step=1), activation="relu"))
    model.add(keras.layers.Dense(1))
    
    model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.R2Score())
    return model

In [20]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(keras.layers.Dense(hp.Int('units_dense0', min_value=36, max_value=50, step=2)))
        if hp.Boolean("relu_dense0"):
            model.add(keras.layers.Activation(keras.activations.relu))
        model.add(keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.01, max_value=0.04, step=0.01)))
        model.add(keras.layers.Dense(18))
        if hp.Boolean("relu_dense1"):
            model.add(keras.layers.Activation(keras.activations.relu))
        model.add(keras.layers.Dense(1))
        model.compile(optimizer=hp.Choice("opti", ['adam', 'RMSprop']), loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.R2Score())
        return model
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [40, 64, 80, 96]),
            **kwargs,
        )


In [21]:
from keras_tuner import Objective
tuner = RandomSearch(
    MyHyperModel(),
    #objective=keras_tuner.Objective('val_r2_score', direction='max'),
    objective = Objective(name="val_r2_score", direction="max"),
    max_trials = 50,
    executions_per_trial = 4,
    tuner_id="13.12.regression.adversary.cgpt"
)

In [22]:
tuner.search(adv_X, adv_y, epochs=200, validation_data=(adv_X_test, adv_y_test), callbacks=[keras.callbacks.EarlyStopping('val_r2_score', patience=30)])

Trial 50 Complete [00h 02m 05s]
val_r2_score: 0.2526484578847885

Best val_r2_score So Far: 0.2885388433933258
Total elapsed time: 00h 52m 58s


In [23]:
models = tuner.get_best_models(num_models=2)

In [29]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_r2_score", direction="max")

Trial 11 summary
Hyperparameters:
units_dense0: 40
relu_dense0: False
dropout_rate: 0.02
relu_dense1: False
opti: adam
batch_size: 40
Score: 0.2885388433933258

Trial 17 summary
Hyperparameters:
units_dense0: 44
relu_dense0: True
dropout_rate: 0.02
relu_dense1: True
opti: adam
batch_size: 64
Score: 0.2639407217502594

Trial 14 summary
Hyperparameters:
units_dense0: 48
relu_dense0: False
dropout_rate: 0.02
relu_dense1: False
opti: adam
batch_size: 40
Score: 0.2615743428468704

Trial 49 summary
Hyperparameters:
units_dense0: 40
relu_dense0: False
dropout_rate: 0.03
relu_dense1: False
opti: RMSprop
batch_size: 40
Score: 0.2526484578847885

Trial 47 summary
Hyperparameters:
units_dense0: 42
relu_dense0: True
dropout_rate: 0.03
relu_dense1: True
opti: adam
batch_size: 96
Score: 0.2372029572725296

Trial 48 summary
Hyperparameters:
units_dense0: 50
relu_dense0: True
dropout_r

In [24]:
best_model = models[0]

In [25]:
best_model.build(input_shape=(None,adv_X.shape[1],))

In [27]:
best_model.layers[1].rate

0.02

In [26]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                800120    
                                                                 
 dropout (Dropout)           (None, 40)                0         
                                                                 
 dense_1 (Dense)             (None, 18)                738       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 800877 (3.06 MB)
Trainable params: 800877 (3.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#X_train, y_train, X_test, y_test, _, _ = data_train_test(train_size=0.7)